# House Rental

## Subject

### Helping to evaluate vacation renting sites

__Short description__: This project is concentrated on the evaluating renting advertisements by studying the electricity consumption in towns where the property is advertised. In the context of users choosing different destinations for vacations, provide them an additional information on those places in terms of environmental conditions. The environmental condition in our limited example is based on the electrical consumption in the destination town.

__Further details__: In France, several house rental web sites have RSS XML flow that can be parsed into a data set containing the list of available rentals. You will find the names of the towns inside the text.

At the same time you have the CSV file from [ENEDIS](https://data.enedis.fr/explore/dataset/consommation-electrique-par-secteur-dactivite-commune/) containing the history of electricity consumption allowing you to estimation how much and for which purpose the energy is consumed. Thus you can provide every community with the “electrical” description, where you will calculate different indicators, such as for example, 
* part/amount of non-resident consumption, which might indicate the importance of the industrial installations in the town.
* evolution of the residencial consumption for several years, which might indicate the growth factor of the town
* evolution of the non-residential consumption
* other indicators left to you

Putting both data sources together allows you to sort/filter the rental advertisements by the “energy” indicators, as, for example, “zero industry” advertisements - quiet locations. To find an exact usage is left to you as a part of the exercise.

## Code

### Fetching results from RSS feed

In [4]:
from lxml import etree
import requests
import sys

def print_titles():
    base = "http://www.ty-gites.com/rss/"
    location = "locations_vacances_"
    regions = ["alsace",
               "aquitaine",
               "auvergne",
               "bourgogne",
               "bretagne",
               "centre_val_de_loire",
               "champagne_ardenne",
               "corse",
               "franche_comte",
               "ile_de_france",
               "languedoc_roussillon",
               "limousin",
               "lorraine",
               "midi_pyrenees",
               "nord_pas_de_calais",
               "normandie",
               "pays_de_la_loire",
               "picardie",
               "poitou_charentes",
               "provence_alpes_cote_d_azur",
               "rhone_alpes",
               "outre_mer"]
    for region in regions:
        url = base+location+region+".xml"
        root = etree.parse(url)
        for item in root.xpath("/rss/channel/item/title"):
            print(item.text.strip('-'))
    return

print_titles()

Chalet  Le Rebberg - Soultzeren - 
Gîte Chez Sylvie LEONHART - SAINT-HIPPOLYTE - 
Gîte Le Panoramic - Sondernach - 
Gîte La Marguerite - Sondernach - 
Gîte Freyburger Jeannine - Elbach - 
Gîte Le Jardin d'Elisa - Niederhaslach - 
Gîte Le nid des hirondelles - KOGENHEIM - 
Gîte kirschner  pierre - Dambach-la-Ville - 
Gîte Wach ''les hirondelles'' *** - Epfig - 
Gîte  Gîte Le Châtaignier *** - EPFIG - 
Gîte  Le Wineck KLUR Michel - KATZENTHAL - 
Appartement Kempf - Stosswihr - 
Gîte Krauffel - Meistratzheim - 
Gîte Patricia Klink - Ungersheim - 
Gîte Gite Zen - Mulhouse - 
Gîte jaeggy - ranspach - 
Chambre d'hôtes LE BOUTON D'OR - MICHEL Charles et Elfriede  - ISSENHAUSEN - 
Gîte LE COQUELICOT - MICHEL Charles et Elfriede - ISSENHAUSEN - 
Gîte LA MARGUERITE - MICHEL Charles et Elfriede - ISSENHAUSEN - 
Gîte LE BLEUET - MICHEL Charles et Elfriede  - ISSENHAUSEN - 
Gîte les tamaris - Saint-Pierre-de-Buzet - 
Chambre d'hôtes DOMAINE TAMBOURIN - BARDOS - 
Gîte Cassetai Chantal - Arsac - 
Gît

Gîte Les Brimbelles - Luvigny - 
Gîte sous les vignes - fameck - 
Chalet Chalet Charlotte - Raon sur Plaine - 
Gîte Gîte Elise - Raon sur Plaine - 
Gîte Clémentine - Raon sur Plaine - 
Studio myappartepinal - Epinal - 
Chalet MICLO DANIEL - ferdrupt - 
Gîte meublé à Nancy-Vandoeuvre - Vandoeuvre-lès-Nancy - 
Gîte De La CHAUME - Le  MENIL - 
Gîte Les Ecureuils - ABRESCHVILLER - 
Appartement St Thiebault - EPINAL - 
Appartement Sweethome-Epinal - EPINAL - 
Chalet chalet avec sauna et bal néo WIFI - Saint-Nabord - 
Gîte BARON - le Clerjus - 
Chambre d'hôtes BARON Monique et Jean-Luc - Le Clerjus - 
Gîte DOMAINE DU HAUT DES BLUCHES - LA BRESSE  - 
Gîte Le Clos des iris - Le Ménil - 
Gîte du Moulin à eau Le Pilan - Portieux - 
Appartement RICHTER Mireille - Gérardmer - 
Gîte GÎTE DE BÉNIPRÉ - Saint-Nabord - 
Gîte Mr MARCHAND Jean Loïc - Gavarnie Gèdre - 
Gîte LE mas des vignes - MOURET - 
Gîte saulodes - villecomtal - 
Gîte Chez Philémon Gite 3* l’Hoste entre Aveyron et Lot - Puylagarde - 
